In [1]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

TensorFlow version: 2.19.0


In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# 기존 shape: (60000, 28, 28)
# tf.newaxis: 새로운 차원 추가 None
# 새로운 shape: (60000, 28, 28, 1)
# cnn등에 적합하게 타입 변경
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [3]:
# numpy -> tf.data.Dataset(데이터 파이프라인)
# 빠른 로딩, 섞기, 배치 처리 제공
# shape: (32, 28, 28, 1), (32,)
train_ds = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [4]:
# Subclassing API: tf.keras.Modeld을 직접 상속받아 새 클래스로 정의 하는 것
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation="relu")
        self.flatten = Flatten()
        self.d1 = Dense(128, activation="relu")
        self.d2 = Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)


# Create an instance of the model
model = MyModel()

In [5]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [6]:
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")

test_loss = tf.keras.metrics.Mean(name="test_loss")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="test_accuracy")

In [7]:
# TensorFlow 그래프로 최적화
@tf.function
def train_step(images, labels):
    # tf.GradientTape()로 자동 미분
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        # 모델 예측
        predictions = model(images, training=True)
        # 손실(loss) 계산
        loss = loss_object(labels, predictions)
    # gradient 계산 + 가중치 업데이트
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [8]:
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    # 예측, 평가모드 종료
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [11]:
EPOCHS = 5

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    # 메트릭 초기화
    train_loss.reset_state()
    train_accuracy.reset_state()
    test_loss.reset_state()
    test_accuracy.reset_state()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
        f"Epoch {epoch + 1}, "
        f"Loss: {train_loss.result()}, "
        f"Accuracy: {train_accuracy.result() * 100}, "
        f"Test Loss: {test_loss.result()}, "
        f"Test Accuracy: {test_accuracy.result() * 100}"
    )

2025-06-29 17:35:16.781672: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-06-29 17:35:18.008821: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1, Loss: 0.1338733285665512, Accuracy: 95.97833251953125, Test Loss: 0.05496769770979881, Test Accuracy: 98.16999816894531


2025-06-29 17:35:47.906550: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 2, Loss: 0.040933653712272644, Accuracy: 98.74500274658203, Test Loss: 0.0517253614962101, Test Accuracy: 98.25
Epoch 3, Loss: 0.02169986255466938, Accuracy: 99.30999755859375, Test Loss: 0.04779838025569916, Test Accuracy: 98.36000061035156


2025-06-29 17:36:48.045022: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 4, Loss: 0.013081563636660576, Accuracy: 99.54499816894531, Test Loss: 0.06931330263614655, Test Accuracy: 97.97000122070312
Epoch 5, Loss: 0.008315094746649265, Accuracy: 99.74666595458984, Test Loss: 0.058724917471408844, Test Accuracy: 98.47999572753906
